In [1]:
import pandas as pd
from ipynb.fs.full.helper_functions import get_scoring_possessions
from ipynb.fs.full.helper_functions import sort_words
from ipynb.fs.full.helper_functions import split_by_team
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.random import set_seed
import datetime
import numpy as np

set_seed(42)
pd.set_option('display.max_rows', 10)

In [2]:
elo_data = pd.read_csv(r"/Users/User/Desktop/Programming/nbaallelo.csv")
poss_1314 = pd.read_excel(r"/Users/User/Desktop/Programming/Possession Data/poss_1314.xlsx")
poss_1314.drop("Unnamed: 0", axis=1, inplace=True)

In [3]:
# Isolating 2013-14 games
elo_data = elo_data.iloc[121054:123513]

In [4]:
# Formatting GameId to match up with possession data
for i, v in enumerate(elo_data.iloc[:, 5]):
    elo_data.iloc[i, 5] = datetime.datetime.strptime(v, '%m/%d/%Y').strftime('%Y/%m/%d')
    elo_data.loc[elo_data.opp_id == 'PHO', 'opp_id'] = 'PHX'
    elo_data.loc[elo_data.opp_id == 'BRK', 'opp_id'] = 'BKN'
    elo_data.loc[elo_data.team_id == 'PHO', 'team_id'] = 'PHX'
    elo_data.loc[elo_data.team_id == 'BRK', 'team_id'] = 'BKN'
    elo_data.iloc[i, 1] = elo_data.iloc[i, 5] + sort_words(elo_data.iloc[i, 8], elo_data.iloc[i, 14])

In [5]:
elo_data = elo_data.sort_values(["team_id", "seasongame"], ascending=[True, True])
elo_data.head()

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,pts,elo_i,elo_n,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
121063,60532,2013/10/30ATLDAL,NBA,1,2014,2013/10/30,1,0,ATL,Hawks,109,1484.9949,1479.5092,38.394112,DAL,Mavericks,118,1521.8522,1527.3379,A,L,0.312638,NaN
121093,60547,2013/11/01ATLTOR,NBA,0,2014,2013/11/01,2,0,ATL,Hawks,102,1479.5092,1485.3594,39.005833,TOR,Raptors,95,1489.1600,1483.3098,H,W,0.627169,NaN
121136,60569,2013/11/03ATLLAL,NBA,1,2014,2013/11/03,3,0,ATL,Hawks,103,1485.3594,1482.3911,38.695332,LAL,Lakers,105,1503.5435,1506.5117,A,L,0.336189,NaN
121168,60585,2013/11/05ATLSAC,NBA,1,2014,2013/11/05,4,0,ATL,Hawks,105,1482.3911,1490.6489,39.559700,SAC,Kings,100,1428.0204,1419.7626,A,W,0.434709,NaN
121193,60597,2013/11/07ATLDEN,NBA,1,2014,2013/11/07,5,0,ATL,Hawks,107,1490.6489,1488.4185,39.326073,DEN,Nuggets,109,1568.9944,1571.2249,A,L,0.263736,NaN


In [6]:
# I noticed here that poss_1314 had one more unique game id than elo_data did, so I found the incorrect game id and removed \
# any rows that had that game id.

to_drop = []

for i, v in enumerate(pd.Series(poss_1314.iloc[:, 7].unique()).isin(pd.Series(elo_data.game_id.unique()))):
    if v == False:
        to_drop.append(i)

to_drop = poss_1314.iloc[:, 7].unique()[to_drop[0]]
poss_1314.drop(poss_1314[poss_1314.iloc[:, 7] == to_drop].index, inplace=True)

In [7]:
elo_by_team = split_by_team(elo_data, elo_data.team_id.unique())
to_add = pd.DataFrame(data={'forecast': 0.938905, 'game_id': '2014/04/16LALSAS', 'game_result': 1}, index=[0])
poss_1314.rename(columns={poss_1314.columns[6]: 'team_id'}, inplace=True)
poss_by_team = split_by_team(poss_1314, poss_1314.iloc[:, 6].unique())

In [8]:
start_prob = []

for i, v in enumerate(elo_by_team):
    v.loc[v.game_result == 'W', 'game_result'] = 1
    v.loc[v.game_result == 'L', 'game_result'] = 0
    start_prob.append(elo_by_team[i][['forecast', 'game_id', 'game_result']])
    
start_prob[26] = pd.concat([start_prob[26], to_add], axis=0)

In [9]:
model_data = []

for i, v in enumerate(start_prob):
    model_data.append(v.merge(poss_by_team[i], 'right', left_on='game_id', right_on=poss_by_team[i].columns[7]))

In [10]:
for i in range(len(model_data)):
    model_data[i].rename(columns={model_data[i].columns[3]: 'Date', model_data[i].columns[4]: 'Opponent',
                                   model_data[i].columns[5]: 'Period', model_data[i].columns[6]: 'Margin',
                                   model_data[i].columns[7]: 'Start Time', model_data[i].columns[8]: 'End Time',
                                   model_data[i].columns[9]: 'Team', model_data[i].columns[10]: 'GameId',
                                   model_data[i].columns[11]: 'Seconds Remaining', model_data[i].columns[12]: 'Poss'},
                          inplace=True)
    model_data[i].reset_index(drop=True, inplace=True)

In [11]:
model_data[0].head()

,forecast,game_id,game_result,Date,Opponent,Period,Margin,Start Time,End Time,Team,GameId,Seconds Remaining,Poss
0,0.312638,2013/10/30ATLDAL,0,2013-10-30,DAL,1,0,12:00,11:35,ATL,2013/10/30ATLDAL,2855,1
1,0.312638,2013/10/30ATLDAL,0,2013-10-30,DAL,1,2,11:35,11:10,ATL,2013/10/30ATLDAL,2830,2
2,0.312638,2013/10/30ATLDAL,0,2013-10-30,DAL,1,0,11:10,11:00,ATL,2013/10/30ATLDAL,2820,1
3,0.312638,2013/10/30ATLDAL,0,2013-10-30,DAL,1,0,11:00,10:52,ATL,2013/10/30ATLDAL,2812,2
4,0.312638,2013/10/30ATLDAL,0,2013-10-30,DAL,1,-2,10:52,10:26,ATL,2013/10/30ATLDAL,2786,1


In [12]:
x_train, x_test, y_train, y_test = [], [], [], []
for i, v in enumerate(model_data):
    x_test.append(v[v.game_id == v.iloc[0, 1]][['forecast', 'Margin', 'Seconds Remaining', 'Poss']])
    x_train.append(v[['forecast', 'Margin', 'Seconds Remaining', 'Poss']].drop(x_test[i].index))
    y_test.append(v[v.game_id == v.iloc[0, 1]]['game_result'])
    y_train.append(v['game_result'].drop(y_test[i].index))
    
x_train = pd.concat(x_train, axis=0)
y_train = pd.concat(y_train, axis=0)

In [13]:
model = Sequential()
model.add(Dense(12, input_shape=(4,), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(np.asarray(x_train).astype(np.float32), np.asarray(y_train).astype(np.float32), epochs=100, batch_size=100)

Epoch 1/100
4596/4596 [==============================] - 5s 921us/step - loss: 1.9567 - accuracy: 0.7135
Epoch 2/100
4596/4596 [==============================] - 4s 935us/step - loss: 0.5035 - accuracy: 0.7266
Epoch 3/100
4596/4596 [==============================] - 4s 911us/step - loss: 0.4961 - accuracy: 0.7333
Epoch 4/100
4596/4596 [==============================] - 4s 917us/step - loss: 0.4882 - accuracy: 0.7435
Epoch 5/100
4596/4596 [==============================] - 4s 914us/step - loss: 0.4811 - accuracy: 0.7494
Epoch 6/100
4596/4596 [==============================] - 4s 951us/step - loss: 0.4763 - accuracy: 0.7556
Epoch 7/100
4596/4596 [==============================] - 4s 918us/step - loss: 0.4736 - accuracy: 0.7569
Epoch 8/100
4596/4596 [==============================] - 4s 927us/step - loss: 0.4720 - accuracy: 0.7583
Epoch 9/100
4596/4596 [==============================] - 4s 923us/step - loss: 0.4699 - accuracy: 0.7599
Epoch 10/100
4596/4596 [==============================]

In [14]:
y_pred, results = [], []

for i in range(30):
    y_pred.append(model.predict(x_test[i]))
    results.append(x_test[i].assign(Probability=y_pred[i]))

7/7 [==============================] - 0s 833us/step


In [15]:
results[0]

,forecast,Margin,Seconds Remaining,Poss,Probability
0,0.312638,0,2855,1,0.409008
1,0.312638,2,2830,2,0.421512
2,0.312638,0,2820,1,0.407962
3,0.312638,0,2812,2,0.369679
4,0.312638,-2,2786,1,0.366366
...,...,...,...,...,...
199,0.312638,-13,41,2,0.000028
200,0.312638,-13,32,1,0.000056
201,0.312638,-11,9,2,0.000112
202,0.312638,-11,5,1,0.000511


The model has trouble predicting the win probabilities at the beginning of the game, but after the first 5-10 possessions the estimation is quite accurate. Another issue the current model has is that the probabilities predicted for teams down by several possessions at the end of the game are slightly too high. This can be seen in the output directly above. It predicts that the team has a probability of winning of 0.003319 when there are 0 seconds remaining and they are down by 9 points. While this estimation is low, it should be 0 or extremely close to 0.

In the coming weeks I will be working on the model and will hopefully be able to correct these small errors.